In [1]:
#!pip install openpyxl

In [2]:
import pandas as pd

In [26]:
test = pd.read_excel('test_queries.xlsx', header =None)

test['question'] = test[0].apply(lambda x : x.split("/")[0])
test['answer'] = test[0].apply(lambda x : x.split("/")[1])
test.drop(columns=[0], inplace=True)

test

,question,answer
0,Are green pants and red shirt a good fit ?,No
1,What do you think about purple shoes and a whi...,No
2,Can I try a brown blouse and a blue skirt?,No
3,Can I combine a red hat with my yellow pants?,No
4,Do you think pink shirt and green pants are go...,No
...,...,...
95,Are green sneaker and red shirt a good fit ?,No
96,What do you think about white pants and a whit...,Yes
97,Can I try a brown hat and a brown purse?,Yes
98,Can I combine a red shirt with my yellow pants?,No


In [4]:
from transformers import pipeline
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

import sys
import keyboard
import json
import ast
import itertools
import numpy as np
from pandas.io.json import json_normalize
import pickle
import ipyplot

model_id = "rasta/distilbert-base-uncased-finetuned-fashion"
classifier = pipeline("text-classification", model=model_id)

def classify(text):
    preds = classifier(text, return_all_scores=True)
    if preds[0][0]['score']  <= preds[0][1]['score']:
        return "Not Fashion"
    else:
        return "Fashion"
    
def attribute_extraction(txt):
    tokenized = sent_tokenize(txt)

    attributes = []
    for i in tokenized:
        wordsList = nltk.word_tokenize(i)
        tagged = nltk.pos_tag(wordsList)

    for i,w in enumerate(tagged) :
        if w[1] in ['NN','NNS','RB'] :
            ind =i 
            attr = w[0]
            while tagged[ind-1][1] in ['JJ','VBN','NN','RB','VBD','EX']:
                    attr = tagged[ind-1][0] + ' ' +  attr
                    ind = ind - 1
                    
            if len(attr.split())==1 and txt.split()[0].lower()=='will':
                attr = tagged[ind-1][0] + ' ' +  attr
                
            if classify(attr) == 'Fashion':
                attributes.append(attr)
            for a in attributes:
                for b in attributes:
                    if (a!=b) and (a in b):
                        attributes.remove(a)
                
            for a in attributes:
                if 'fit' in a :
                    attributes = list(map(lambda x: x.replace(a, a.replace(' fit','')), attributes))
                if 'match' in a :  
                    attributes = list(map(lambda x: x.replace(a, a.replace(' match','')), attributes))                                       
                
    return attributes

In [5]:
from simcse import SimCSE

model_SIMCSE = SimCSE("princeton-nlp/sup-simcse-roberta-large")

with open('index.pkl', 'rb') as f:
    index = pickle.load(f)
    
from nltk.stem import WordNetLemmatizer
  
lemmatizer = WordNetLemmatizer()

model_SIMCSE.index = index
items = index['sentences']

def similar_items(attr):
    similar_items=[]
    results = model_SIMCSE.search(attr,top_k=10,threshold=0.779)
    for i in range(len(results)):
        similar_items.append(results[i][0])
    return similar_items

matrix = pd.read_csv('Final_co-occurence_polyvore_Adel.csv')

def matrix_search_advice(attr,k):
    match = []
    i = 0
    append = True
    
    for a in matrix['bigram'] :
        if attr in a :
            a = tuple(a[1:-1].replace('\'','').split(", "))

            if attr in a[0] :
                wrd = a[1]
            else:
                wrd = a[0]
                
            remove = False
            if (not wrd in match) and (not attr in wrd): 
                        match.append(wrd)
                        i = i + 1
            if i == k:
                break
                
    return match,i

def matrix_search_match(attr,k):
    match = []
    i = 0
    append = True
    
    for a in matrix['bigram'] :
        if attr in a :
            a = tuple(a[1:-1].replace('\'','').split(", "))

            if attr in a[0] :
                wrd = a[1]
            else:
                wrd = a[0]
                
            remove = False
            if (not wrd in match) and (not attr in wrd): 
                    for el in match : 
                        if model_SIMCSE.similarity(el,wrd) > 0.7:
                            remove = True
                    if not remove:
                        match.append(wrd)
                        i = i + 1
            #if i == k+3:
            if i == k:
                break

    return match,i

def garment_matching(attr,k):           # Returns k best matches to the given attribute
    
    attr = " ".join([lemmatizer.lemmatize(i) for i in attr.split()])
    i= 0
    match = []

    if attr in items:
        if k == 5 :
            match,i = matrix_search_match(attr,k)
        if k == 10 :
            match,i = matrix_search_advice(attr,k)

    else :
        similar = similar_items(attr)
        stop = False
        ind = 0
        while (not stop) and (ind < len(similar)):
            print(len(similar))
            if similar[ind] in items:
                if k == 5:
                    match,i = matrix_search_match(similar[ind],k)
                if k == 10:
                    match,i = matrix_search_advice(similar[ind],k)
                if (i>0):
                    stop = True 
                    attr = similar[ind]
            ind = ind + 1

    if (i==0):
        message = 'This attribute was not found for the garment matching try another attribute!'
        return message,[]
        
    return attr,match

def garment_advice(attr1 , attr2, k=10):
    match = []
    
    i = 0
    attr1, match = garment_matching(attr1,k)
        
    if match is None :
        return False
    
    if attr2 in match:
        return True
    else:
        for el in match:
            if model_SIMCSE.similarity(el,attr2) > 0.9 :
                return True
    
    return False

### Check for any problem before entering the pipeline

In [6]:
for i in range (100):
    if len(attribute_extraction(test["question"][i])) != 2 :
        print("Problem at line ",i ,": ",  test["question"][i])


Problem at line  27 :  Will gold top be a good match with a black purse ?  
Problem at line  51 :  Is a white necklace worn with a black hat? ? 
Problem at line  70 :  Will red jacket fits with black heel? 


### Script to evaluate the queries

In [28]:
answers = []
for i in range (100):
    attr = attribute_extraction(test["question"][i])
    b = garment_advice(attr[0],attr[1])
    if b==True :
        answers.append(" Yes")
    else :
        answers.append(" No")

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.37it/s]


10


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  8.06it/s]


10
10


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.29it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.72it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.88it/s]


10


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.29it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.76it/s]


## Calculate the metric

In [29]:
accuracy = 0
for i in range(100):
    if answers[i] == test['answer'][i]:
        accuracy = accuracy + 1

print("Accuracy of the model : ", accuracy, " %")

Accuracy of the model :  83  %
